In [1]:
# USAGE
# python text_detection.py --image images/lebron_james.jpg --east frozen_east_text_detection.pb

# import the necessary packages
from imutils.object_detection import non_max_suppression
#for identifying bounding box
import numpy as np
import argparse
import time
import cv2



In [2]:
import pytesseract
#for text recognition

In [3]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\Sid\AppData\Local\Tesseract-OCR\tesseract.exe'

In [4]:
net = cv2.dnn.readNet("frozen_east_text_detection.pb")
#pretraiined neural net via east 

In [21]:
im_to_text = []
#image given here to process
def text_detector(image):
    orig  = image
    (H,W) = image.shape[:2]
    (newW,newH) = (320,320)
    rW = W/float(newW)
    rH = H/float(newH)
    image = cv2.resize(image,(newW,newH))
    (H,W)= image.shape[:2]
    layerNames = ["feature_fusion/Conv_7/Sigmoid","feature_fusion/concat_3"]
    blob = cv2.dnn.blobFromImage(image,1.0,(W,H),(123.68,116.78,103.94),swapRB=True,crop=False)
    net.setInput(blob)
    (scores,geometry) = net.forward(layerNames)
    (numRows,numCols)= scores.shape[2:4]
    rects= []
    confidences = []
    
    for y in range(0, numRows):
        scoresData = scores[0,0,y]
        xData0 = geometry[0,0,y]
        xData1 = geometry[0,1,y]
        xData2 = geometry[0,2,y]
        xData3 = geometry[0,3,y]
        anglesData = geometry[0,4,y]
    
        for x in range(0,numCols):
            if scoresData[x]<0.5:
                continue
            (offsetX,offsetY) = (x*4.0,y*4.0)
            angle = anglesData[x]
            cos = np.cos(angle)
            sin = np.sin(angle)
            h = xData0[x] + xData2[x]
            w = xData1[x] + xData3[x]
            #text prediction bounding box
            endX = int(offsetX + (cos*xData1[x])+ (sin*xData2[x]))
            endY = int(offsetY - (sin*xData1[x])+ (cos*xData2[x]))

            startX = int(endX-w)
            startY = int(endY-h)
            #add bounding box to list
            rects.append((startX,startY,endX,endY))
            confidences.append(scoresData[x])

        boxes = non_max_suppression(np.array(rects),probs = confidences)
        #this is where we identify the bounding boxes
        #once we know where bounding boxes are we define a box where text is 
        #ROI -region of interest- bounding box
        #startx,starty, endx,endy -> start left top coodinate, end bottom right coordinate
        for(startX,startY,endX,endY) in boxes:
            startX = int(startX*rW)
            startY = int(startY*rH)

            endX = int(endX*rW)
            endY = int(endY*rH)
            #this is an adjustable metric for improving accuracy
            boundary = 5
            #boundary is given to the box so that some information has some background of text
            #else tesseract wont work properly
            text = orig[startY-boundary:endY+ boundary,startX-boundary:endX+boundary]
            #text is not a string,but an image vector
            text = cv2.cvtColor(text.astype(np.uint8),cv2.COLOR_BGR2GRAY)
            #this color image vector is converted to a gray  image for text recognition
            textRecognized = pytesseract.image_to_string(text)
            #the image is fairly visible now and the text can be fed into pytesseract for text recognition
            im_to_text.append(textRecognized)
            cv2.rectangle(orig,(startX,startY),(endX,endY),(0,255,0))
            #draw the bounding box on the originial image 
            processed= cv2.putText(orig,textRecognized,(endX,endY+5),cv2.FONT_HERSHEY_SIMPLEX,1,
                              (0, 0, 255), 2, cv2.LINE_AA)
            #print the text onto the bounding box that we created in the last step 
            #0,0,255 is color and  2 is thickness , cv2.LINE_AA -> type of line we use
    print(im_to_text)        
    return processed
        
        
    

In [22]:
image0 = cv2.imread(r'C:\Users\Sid\Documents\ComputerVision\images\car_wash.png')
image1 = cv2.imread(r'C:\Users\Sid\Documents\ComputerVision\images\lebron_james.jpg')
image2 = cv2.imread(r'C:\Users\Sid\Documents\ComputerVision\images\sign.jpg')
image3 = cv2.imread(r'C:\Users\Sid\Documents\ComputerVision\images\mug.jpeg')
image4 = cv2.imread(r'C:\Users\Sid\Documents\ComputerVision\images\random.JPG')
image5 = cv2.imread(r'C:\Users\Sid\Documents\ComputerVision\images\peanutbutter.JPEG')

In [23]:
#check if image is none, will cause error if image size is none
image5

array([[[ 11,  16,  19],
        [ 11,  16,  19],
        [  9,  16,  19],
        ...,
        [ 86, 107, 134],
        [ 86, 107, 134],
        [ 86, 107, 134]],

       [[ 11,  16,  19],
        [ 11,  16,  19],
        [  9,  16,  19],
        ...,
        [ 86, 107, 134],
        [ 86, 107, 134],
        [ 86, 107, 134]],

       [[ 11,  15,  20],
        [ 11,  15,  20],
        [  9,  15,  20],
        ...,
        [ 86, 107, 134],
        [ 86, 107, 134],
        [ 86, 107, 134]],

       ...,

       [[129, 151, 179],
        [129, 151, 179],
        [129, 151, 179],
        ...,
        [115, 138, 164],
        [115, 138, 164],
        [115, 138, 164]],

       [[129, 151, 179],
        [129, 151, 179],
        [129, 151, 179],
        ...,
        [115, 138, 164],
        [115, 138, 164],
        [115, 138, 164]],

       [[129, 151, 179],
        [129, 151, 179],
        [129, 151, 179],
        ...,
        [115, 138, 164],
        [115, 138, 164],
        [115, 138, 164]]

In [24]:
#array = [image0,image1,image2,image3,image4]
array = [image5]

In [25]:
#loop the image 2 times, in first step the image is resized to multiple of 32 and interpolation by area
#once again the resize is done to compare 
for i in range(0,2):
    for img in array:
        imageO = cv2.resize(img, (640,320) , interpolation= cv2.INTER_AREA)
        orig = cv2.resize(img, (640,320) , interpolation= cv2.INTER_AREA)
        textDetected = text_detector(imageO)
        #identifies the bounding box and gets back the processed image
        cv2.imshow("Orig image",orig)
        cv2.imshow("Text Detection",textDetected)
        time.sleep(2)
        #to understand both images
        k = cv2.waitKey(30)
        #important to put this, else imshow function wont work
        if k==27:
            break
cv2.destroyAllWindows()
        

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [13]:
import numpy as np 
  
# function to get unique values 
def unique(list1): 
    x = np.array(list1) 
    print(np.unique(x)) 

In [14]:
unique(im_to_text)

['']


In [26]:
image5.shape

(1600, 1600, 3)

In [29]:
height,width,_ = image5.shape

In [30]:
roi = image5[0:height, 400:width]

In [31]:
cv2.imshow("roi",roi)